In [1]:
import tensorflow as tf
import librosa
import os
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from scipy.io.wavfile import write

In [3]:
import pandas as pd
df_test_metadata = pd.read_csv('./../test_vox1_metadata.csv')
images = {}
for row in df_test_metadata.iterrows():
    row_item = row[1]
    user_id = row_item['VoxCeleb1 ID']
    images[user_id] = {
        'fullpath': row_item['full_paths'].replace('spmel_test','wavtest').replace('_0','/0')[:-4]+'.wav',
        'user_id': user_id,
        }

In [4]:
import itertools
image_combinations = list(map(dict, itertools.combinations(
    images.items(), 2)))

In [5]:
tf.__version__

'1.15.0'

In [6]:
# Reads wav file and produces spectrum
# Fourier phases are ignored
N_FFT = 2048
def read_audio_spectum(filename):
    x, fs = librosa.load(filename)
    S = librosa.stft(x, N_FFT)
    p = np.angle(S)
    
    S = np.log1p(np.abs(S[:,:430]))  
    return S, fs

### Load style and content

In [7]:
# CONTENT_FILENAME = "inputs/imperial.mp3"
# STYLE_FILENAME = "inputs/usa.mp3"

In [8]:
# a_content, fs = read_audio_spectum(CONTENT_FILENAME)
# a_style, fs = read_audio_spectum(STYLE_FILENAME)

# N_SAMPLES = a_content.shape[1]
# N_CHANNELS = a_content.shape[0]
# a_style = a_style[:N_CHANNELS, :N_SAMPLES]

In [9]:
# display(Audio(CONTENT_FILENAME))
# display(Audio(STYLE_FILENAME))

In [10]:
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# plt.title('Content')
# plt.imshow(a_content[:400,:])
# plt.subplot(1, 2, 2)
# plt.title('Style')
# plt.imshow(a_style[:400,:])
# plt.show()

### Visualize spectrograms for content and style tracks

### Compute content and style feats

In [31]:
from sys import stderr

def audio_style_transfer(content_filename, style_filename, content_id, style_id):
    a_content, fs = read_audio_spectum(os.path.join('./../',content_filename))
    a_style, fs = read_audio_spectum(os.path.join('./../',style_filename))

    N_SAMPLES = a_content.shape[1]
    N_CHANNELS = a_content.shape[0]
    a_style = a_style[:N_CHANNELS, :N_SAMPLES]
    if(a_style.shape[0] < N_CHANNELS or a_style.shape[1] < N_SAMPLES):
        result_style = np.zeros_like(a_content)
        result_style[:a_style.shape[0], :a_style.shape[1]] = a_style
        a_style = result_style
    N_FILTERS = 4096

    a_content_tf = np.ascontiguousarray(a_content.T[None,None,:,:])
    a_style_tf = np.ascontiguousarray(a_style.T[None,None,:,:])

    # filter shape is "[filter_height, filter_width, in_channels, out_channels]"
    std = np.sqrt(2) * np.sqrt(2.0 / ((N_CHANNELS + N_FILTERS) * 11))
    kernel = np.random.randn(1, 11, N_CHANNELS, N_FILTERS)*std

    g = tf.Graph()
    with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
        # data shape is "[batch, in_height, in_width, in_channels]",
        x = tf.placeholder('float32', [1,1,N_SAMPLES,N_CHANNELS], name="x")

        kernel_tf = tf.constant(kernel, name="kernel", dtype=tf.float32)
        conv = tf.nn.conv2d(
            x,
            kernel_tf,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv")

        net = tf.nn.relu(conv)
        content_features = net.eval(feed_dict={x: a_content_tf})
        style_features = net.eval(feed_dict={x: a_style_tf})

        features = np.reshape(style_features, (-1, N_FILTERS))
        style_gram = np.matmul(features.T, features) / N_SAMPLES


    ALPHA= 1e-2
    learning_rate= 1e-3
    iterations = 100

    result = None
    with tf.Graph().as_default():

        # Build graph with variable input
    #     x = tf.Variable(np.zeros([1,1,N_SAMPLES,N_CHANNELS], dtype=np.float32), name="x")
        x = tf.Variable(np.random.randn(1,1,N_SAMPLES,N_CHANNELS).astype(np.float32)*1e-3, name="x")

        kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
        conv = tf.nn.conv2d(
            x,
            kernel_tf,
            strides=[1, 1, 1, 1],
            padding="VALID",
            name="conv")


        net = tf.nn.relu(conv)

        content_loss = ALPHA * 2 * tf.nn.l2_loss(
                net - content_features)

        style_loss = 0

        _, height, width, number = map(lambda i: i.value, net.get_shape())

        size = height * width * number
        feats = tf.reshape(net, (-1, number))
        gram = tf.matmul(tf.transpose(feats), feats)  / N_SAMPLES
        style_loss = 2 * tf.nn.l2_loss(gram - style_gram)

         # Overall loss
        loss = content_loss + style_loss

        opt = tf.contrib.opt.ScipyOptimizerInterface(
              loss, method='L-BFGS-B', options={'maxiter': 300})

        # Optimization
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())

            print('Started optimization.')
            opt.minimize(sess)

            print('Final loss:', loss.eval())
            result = x.eval()
    a = np.zeros_like(a_content)
    a[:N_CHANNELS,:] = np.exp(result[0,0].T) - 1

    # This code is supposed to do phase reconstruction
    p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
    for i in range(500):
        S = a * np.exp(1j*p)
        x = librosa.istft(S)
        p = np.angle(librosa.stft(x, N_FFT))

    OUTPUT_FILENAME = f'{content_id}x{style_id}'+'.wav'
    write(OUTPUT_FILENAME, fs, x)
    #librosa.output.write_wav(OUTPUT_FILENAME, x, fs)

In [ ]:
for pair in image_combinations:
    content_id, style_id = pair.keys()
    userpair = f'{content_id}x{style_id}'
    if content_id == 'id10270':
        continue
    print(userpair)
#     try:
    audio_style_transfer(pair[content_id]['fullpath'],pair[style_id]['fullpath'], content_id, style_id)
#     except:
#         print(f'Skipped:', userpair)

id10271xid10272
Started optimization.
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 344.412415
  Number of iterations: 300
  Number of functions evaluations: 309
Final loss: 344.4124
id10271xid10273
Started optimization.
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 190.554367
  Number of iterations: 300
  Number of functions evaluations: 315
Final loss: 190.55437
id10271xid10274
Started optimization.
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 374.487396
  Number of iterations: 300
  Number of functions evaluations: 322
Final loss: 374.4874
id10271xid10275
Started optimization.
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 179.211884
  Number of

In [8]:
# N_FILTERS = 4096

# a_content_tf = np.ascontiguousarray(a_content.T[None,None,:,:])
# a_style_tf = np.ascontiguousarray(a_style.T[None,None,:,:])

# # filter shape is "[filter_height, filter_width, in_channels, out_channels]"
# std = np.sqrt(2) * np.sqrt(2.0 / ((N_CHANNELS + N_FILTERS) * 11))
# kernel = np.random.randn(1, 11, N_CHANNELS, N_FILTERS)*std
    
# g = tf.Graph()
# with g.as_default(), g.device('/cpu:0'), tf.Session() as sess:
#     # data shape is "[batch, in_height, in_width, in_channels]",
#     x = tf.placeholder('float32', [1,1,N_SAMPLES,N_CHANNELS], name="x")

#     kernel_tf = tf.constant(kernel, name="kernel", dtype=tf.float32)
#     conv = tf.nn.conv2d(
#         x,
#         kernel_tf,
#         strides=[1, 1, 1, 1],
#         padding="VALID",
#         name="conv")
    
#     net = tf.nn.relu(conv)

#     content_features = net.eval(feed_dict={x: a_content_tf})
#     style_features = net.eval(feed_dict={x: a_style_tf})
    
#     features = np.reshape(style_features, (-1, N_FILTERS))
#     style_gram = np.matmul(features.T, features) / N_SAMPLES

### Optimize

In [9]:
# from sys import stderr

# ALPHA= 1e-2
# learning_rate= 1e-3
# iterations = 100

# result = None
# with tf.Graph().as_default():

#     # Build graph with variable input
# #     x = tf.Variable(np.zeros([1,1,N_SAMPLES,N_CHANNELS], dtype=np.float32), name="x")
#     x = tf.Variable(np.random.randn(1,1,N_SAMPLES,N_CHANNELS).astype(np.float32)*1e-3, name="x")

#     kernel_tf = tf.constant(kernel, name="kernel", dtype='float32')
#     conv = tf.nn.conv2d(
#         x,
#         kernel_tf,
#         strides=[1, 1, 1, 1],
#         padding="VALID",
#         name="conv")
    
    
#     net = tf.nn.relu(conv)

#     content_loss = ALPHA * 2 * tf.nn.l2_loss(
#             net - content_features)

#     style_loss = 0

#     _, height, width, number = map(lambda i: i.value, net.get_shape())

#     size = height * width * number
#     feats = tf.reshape(net, (-1, number))
#     gram = tf.matmul(tf.transpose(feats), feats)  / N_SAMPLES
#     style_loss = 2 * tf.nn.l2_loss(gram - style_gram)

#      # Overall loss
#     loss = content_loss + style_loss

#     opt = tf.contrib.opt.ScipyOptimizerInterface(
#           loss, method='L-BFGS-B', options={'maxiter': 300})
        
#     # Optimization
#     with tf.Session() as sess:
#         sess.run(tf.initialize_all_variables())
       
#         print('Started optimization.')
#         opt.minimize(sess)
    
#         print('Final loss:', loss.eval())
#         result = x.eval()
    
#     a = np.zeros_like(a_content)
#     a[:N_CHANNELS,:] = np.exp(result[0,0].T) - 1

#     # This code is supposed to do phase reconstruction
#     p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
#     for i in range(500):
#         S = a * np.exp(1j*p)
#         x = librosa.istft(S)
#         p = np.angle(librosa.stft(x, N_FFT))
#     OUTPUT_FILENAME = f'{content_id}x{style_id}'
#     write(OUTPUT_FILENAME, fs, x)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Started optimization.
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
  Objective function value: 529.248230
  Number of iterations: 300
  Number of functions evaluations: 317
Final loss: 529.2482


### Invert spectrogram and save the result

In [23]:
# a = np.zeros_like(a_content)
# a[:N_CHANNELS,:] = np.exp(result[0,0].T) - 1

# # This code is supposed to do phase reconstruction
# p = 2 * np.pi * np.random.random_sample(a.shape) - np.pi
# for i in range(500):
#     S = a * np.exp(1j*p)
#     x = librosa.istft(S)
#     p = np.angle(librosa.stft(x, N_FFT))

# OUTPUT_FILENAME = 'outputs/out.wav'
# #librosa.output.write_wav(OUTPUT_FILENAME, x, fs)

In [ ]:
# len(x)

In [25]:
# write(OUTPUT_FILENAME, fs, x)

In [24]:
# print(OUTPUT_FILENAME)
# display(Audio(x, rate=fs))

outputs/out.wav


### Visualize spectrograms

In [ ]:
# plt.figure(figsize=(15,5))
# plt.subplot(1,3,1)
# plt.title('Content')
# plt.imshow(a_content[:400,:])
# plt.subplot(1,3,2)
# plt.title('Style')
# plt.imshow(a_style[:400,:])
# plt.subplot(1,3,3)
# plt.title('Result')
# plt.imshow(a[:400,:])
# plt.show()